In [75]:
import numpy as np

In [76]:
DIM = 10

In [77]:
class Player():
    """Class representing a player in a game of battleship."""
    def __init__(self, name="dummy"):
        """a player object is initialized with the name of the player"""
        self.player_name = name
        self.seamap = Map()
        self.enemymap = Map()
        self.fleet = Fleet()
    
    def display_map(self):
        """method to visualize player's map"""
        self.seamap.display()
    
    def place_ships(self):
        """before the game start this method places the ships"""
        for key, ship in self.fleet.ships.items():
            while not ship.valid_position():
                xloc = int(input(key + " x location "))
                yloc = int(input(key + " y location "))
                
                orientation = input(key + " orientation ") 
                ship.set_location((xloc, yloc))
                ship.set_orientation(orientation)
                self.seamap.place_ship(ship)
                self.display_map()
    
    def shoot(self):
        """don't forget a docstring"""
        pass
    
    def evaluate(self, other_player_map):
        """don't forget a docstring"""
        pass
    
    def record(self):
        """don't forget a docstring"""
        pass
    
    def has_lost(self):
        return self.fleet.destroyed()

In [78]:
class Map():
    def __init__(self):
        self.karte = np.zeros(shape=(DIM, DIM), dtype=int)
        populated_cells = {}
        
    def place_ship(self, ship):
        x = tmp_x = ship.get_np_x()
        y = tmp_y = ship.get_np_y()
        tmp_list = []
        for i in range(ship.size):
            if ship.get_orientation() == 'v':
                tmp_x = x + i
            else:
                tmp_y = y + i
            tmp_list.append((tmp_x, tmp_y))
    
        try:
            if Map.lists_overlap(tmp_list, self.get_populated_cell_coordinates()):
                raise Exception('Ship positions overlap')
            else:
                for tmp_x, tmp_y in tmp_list:
                    self.populated_cells[(tmp_x, tmpy)] = ship.name
                    self.karte[tmp_x][tmp_y] = 1
        except Exception as inst:
            print(type(inst)) 
            
    @classmethod
    def lists_overlap(cls, lst1, lst2):
        return bool([value for value in lst1 if value in lst2])
            
    def display(self):
        print(self.karte)
        
    def get_populated_cell_coordinates(self):
        return self.populated_cells.keys()
    
        

In [79]:
class Location():
    def __init__(self, coordinates):
        self.x_location = coordinates[0] - 1
        self.y_location = coordinates[1] - 1
        
    def set_np_x(self, x_coordinate):
        self.x_location = x_coordinaterdinate - 1
    
    def set_np_y(self, y_coordinate):
        self.y_location = y_coordinate - 1
    
    def get_np_x(self):
        return self.x_location
    
    def get_np_y(self):
        return self.y_location
    
    def get_x_coordinate(self):
        return (self.x_location + 1)
    
    def get_y_coordinate(self):
        return (self.y_location + 1)
    
    def is_valid(self):
        is_valid = False
        if (0 <= self.get_np_x() <= DIM - 1) and (0 <= self.get_np_y() <= DIM - 1):
            is_valid = True
        return is_valid
    
class Orientation():
    def __init__(self, orientation_str):
        if orientation_str.lower()[0] == 'v':
            self.orientation = 'v'
        elif orientation_str.lower()[0] == 'h':
            self.orientation = 'h'
            
    def get_orientation(self):
        return self.orientation
            
    def is_valid(self):
        return (self.orientation == 'v' or self.orientation == 'h')

class Position():
    def __init__(self, location=None, orientation=None):
        self.location = Location(location)
        self.orientation = Orientation(orientation)
        
    def set_location(self, coordinates):
        self.location = Location(coordinates)
        
    def set_orientation(self, orientation):
        self.orientation = Orientation(orientation)
        
    def is_valid(self):
        return (self.location.is_valid() and self.orientation.is_valid())

In [80]:
class Ship():
    """Class representing a ship in a game of battleship."""
    def __init__(self, name=None, size=None):
        self.size = size
        self.name = name
        self.hits = [0] * self.size
        self.position = Position((-1, -1), 'x')
        
    def set_orientation(self, orientation_str):
        self.position.set_orientation(orientation_str)
        
    def set_location(self, coordinates):
        self.position.set_location(coordinates)
        
    def destroyed(self):
        return (self.hits.count(0) == 0)
        
    def get_np_x(self):
        return self.position.location.get_np_x()
    
    def get_np_y(self):
        return self.position.location.get_np_y()
    
    def get_orientation(self):
        return self.position.orientation.get_orientation()
    
    def valid_position(self):
        return self.position.is_valid()

In [81]:
class Fleet():
#    ship_properties = [('carrier', 5),\
#                       ('battleship', 4),\
#                       ('cruiser', 3),\
#                       ('submarine', 3),\
#                       ('destroyer', 2)]
    ship_properties = [('carrier', 5), ('battleship', 4)]
    def __init__(self):
        self.ships = {}
        for properties in Fleet.ship_properties:
            self.ships[properties[0]] = Ship(properties[0], properties[1])
            
            
    def destroyed(self):
        destroyed = True
        for name, ship in self.ships.items():
            if not ship.destroyed():
                destroyed = False
        
        return destroyed
    

In [82]:
player1 = Player('Alice')
player1.place_ships()

carrier x location 1
carrier y location 2
carrier orientation v
<class 'AttributeError'>
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
battleship x location 1
battleship y location 2
battleship orientation v
<class 'AttributeError'>
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [14]:
def main():
    # create players
    player1 = Player('Alice')
    player2 = Player('Bob')
    
    # place ships
    player1.place_ships()
    player2.place_ships()
    
    while (not player1.has_lost()) and (not player2.has_lost()):
        # game loop
        # player 1: shoot
        # player 1: evaluate
        player1.evaluate(player2.seamap)
        # player 1: record
        # player 2: shoot
        # player 2: evaluate
        # player 2: record
        
    if player1.has_lost():
        print("player 2 wins")
    else:
        print("player 1 wins")

In [24]:
 var = input("name")

namehallo


In [25]:
print(var)

hallo
